In [12]:
import librosa
from pydub import AudioSegment
from pydub.effects import compress_dynamic_range, normalize
import pydub.scipy_effects as scipy_effects
from pydub.playback import play
import soundfile as sf

import os
from pydub import AudioSegment
import json
import tqdm
import yaml

ROOT = r'D:\Download\steam\steam\steamapps\common\Left 4 Dead 2\left4dead2'
VOICE_ROOT = os.path.join(ROOT, 'sound', 'player', 'survivor', 'voice')
CHARACTERS = ['coach', 'gambler', 'mechanic', 'producer']

with open('../config/voice.yaml', 'r', encoding='utf-8') as fp:
    config = yaml.load(fp, Loader=yaml.FullLoader)

def read_json(path: str) -> dict:
    with open(path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
    return data

def write_json(path: str, obj: dict):
    with open(path, 'w', encoding='utf-8') as fp:
        json.dump(obj, fp, ensure_ascii=False, indent=4)

def speed_change(sound: AudioSegment, speed:float = 1.0) -> AudioSegment:
    # Manually override the frame_rate. This tells the computer how many
    # samples to play per second
    sound_with_altered_frame_rate = sound._spawn(sound.raw_data, overrides={
         "frame_rate": int(sound.frame_rate * speed)
      })
     # convert the sound with altered frame rate to a standard frame rate
     # so that regular playback programs will work right. They often only
     # know how to play audio at standard frame rate (like 44.1k)
    return sound_with_altered_frame_rate.set_frame_rate(sound.frame_rate)

In [16]:
def align_time_one(name: str, wav_file: str) -> AudioSegment:
    meta_data = read_json(f'../transcription/{name}.meta.json')
    result_path = os.path.join('..', 'dist', config[name]['mod_name'], 'sound', 'player', 'survivor', 'voice', name, wav_file)

    meta_info = meta_data[wav_file]
    length = meta_info['length']

    result_audio: AudioSegment = AudioSegment.from_file(result_path)
    result_length = len(result_audio)

    speed_ratio = result_length / length

    speed_up_audio = speed_change(result_audio, speed_ratio)

    print(length / result_length)

    print("expect {}, actual {}, now {}".format(length, result_length, speed_up_audio.__len__()))
    return speed_up_audio
    

align_time_one('coach', 'worldc2m3b03.wav')

0.8307383224510296
expect 3308, actual 3982, now 3308
